In [ ]:
import os
import numpy as np
import tensorflow as tf
import face_recognition as fr
import cv2

In [ ]:
model_save_path = r'D:\FIAP\2024\1_Semestre\AI ENGENEERING, COGNITIVE AND SEMANTIC COMPUTATION & IOT\CP\KenAI\Reconhecimento_pessoa\modelo.h5'
modelo = tf.keras.models.load_model(model_save_path)

In [ ]:
# Função para reconhecimento facial ao vivo
def live_face_recognition(model, cap):
    while True:
        ret, frame = cap.read()  # Captura um frame da webcam
        if not ret:
            break

        # Converter o frame para RGB para o face_recognition
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Encontrar as localizações dos rostos no frame usando face_recognition
        face_locations = fr.face_locations(rgb_frame)

       # Pré-processar cada rosto e calcular os face encodings
        for (top, right, bottom, left) in face_locations:
            face_img = rgb_frame[top:bottom, left:right]  # Recortar o rosto
            
            # Redimensionar o rosto para o tamanho esperado pelo modelo (32x32)
            resized_face_img = cv2.resize(face_img, (32, 32))
            
            # Converter a imagem para o formato adequado (32, 32, 3)
            resized_face_img = cv2.cvtColor(resized_face_img, cv2.COLOR_RGB2BGR)  # Se necessário
            
            # Passar as coordenadas do rosto para face_encodings
            face_encodings = fr.face_encodings(rgb_frame, [(top, right, bottom, left)])  # Obter o encoding do rosto
            if face_encodings:
                face_encoding = face_encodings[0]

                # Adicionar dimensão de lote
                face_encoding = np.expand_dims(face_encoding, axis=0)

                # Fazer previsão usando o modelo
                predictions = model.predict(resized_face_img)  # Passar a imagem redimensionada
                predicted_class = np.argmax(predictions)

                # Definir a cor e o rótulo com base na classe prevista
                color = (0, 255, 0) if predicted_class == 0 else (0, 0, 255)  # Verde para reconhecido, vermelho para não reconhecido
                label = "Larissa" if predicted_class == 0 else "Outra pessoa"

                # Desenhar o retângulo ao redor do rosto detectado na webcam e escrever o nome
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        scale_percent = 50  # Percentual do tamanho original
        width = int(frame.shape[1] * scale_percent / 200)
        height = int(frame.shape[0] * scale_percent / 200)
        resized_frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)

        # Mostrar o frame com o resultado do reconhecimento facial em tempo real
        cv2.imshow('Live Face Recognition', resized_frame)

        # Pressione 'esc' para sair do loop
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Liberar a webcam e fechar a janela
    cap.release()
    cv2.destroyAllWindows()

# Inicializar o objeto de captura de vídeo
cap = cv2.VideoCapture(0)

# Verificar se a webcam foi aberta corretamente
if not cap.isOpened():
    print("Erro ao abrir a webcam")
    exit()

# Chamar a função de reconhecimento facial ao vivo
live_face_recognition(modelo, cap)